In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow import keras
from scipy.stats import norm
from numpy.polynomial.hermite import hermgauss

import matplotlib.pyplot as plt

import urllib
import os

np.random.seed(42)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2024-04-29 10:57:59.614828: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
nr_samples_surface_plot = 21
nr_samples_scatter_plot = 1000
nr_samples_error_calculation = 10000

In [3]:
# Model parameters. Re-train model after any changes.
s_min_interest = 25
s_max_interest = 150
t_min_interest = 0.5
t_max_interest = 4.

riskfree_rate_min = 0.1
riskfree_rate_max = 0.3
riskfree_rate_eval = 0.2

volatility_min = 0.1
volatility_max = 0.3
volatility1_eval = 0.1
volatility2_eval = 0.3

correlation_min = 0.2
correlation_max = 0.8
correlation_eval = 0.5

strike_price = 100.

In [4]:
nr_nodes_per_layer = 90
initial_learning_rate = 0.001
localisation_parameter = 1/10.

n_train = 10000
nr_epochs = 601

In [5]:
dimension_state = 2
dimension_parameter = 4
dimension_total = 1 + dimension_state + dimension_parameter

t_min = 0.
t_max = t_max_interest
s_max = strike_price * (1 + 3*volatility_max*t_max)
x_max = np.log(s_max)
x_min = 2*np.log(strike_price) - x_max

normalised_max = 1
normalised_min = -1

def transform_ab_to_cd(x, a, b, c, d): 
    """
    Perform a linear transformation of a scalar from the souce interval
    to the target interval.

    Keyword arguments:
    x -- scalar point(s) to transform
    a, b -- interval to transform from
    c, d -- interval to transform to 
    """
    return c + (x-a) * (d-c) / (b-a)

def transform_to_logprice(x): 
    """ Transform normalised variable to the log-price. """
    return transform_ab_to_cd(x, normalised_min, normalised_max, x_min, x_max)

def transform_to_time(t): 
    """ Transform normalised variable to the time variable. """
    return transform_ab_to_cd(t, normalised_min, normalised_max, t_min, t_max)

def normalise_logprice(x):
    """ Transform log-price to its corresponding normalised variable. """
    return transform_ab_to_cd(x, x_min, x_max, normalised_min, normalised_max)

def normalise_time(t): 
    """ Transform time to its corresponding normalised variable. """
    return transform_ab_to_cd(t, t_min, t_max, normalised_min, normalised_max)

t_min_interest_normalised = normalise_time(t_min_interest)
t_max_interest_normalised = normalise_time(t_max_interest)

diff_dx = (normalised_max-normalised_min) / (x_max-x_min) 
diff_dt = (normalised_max-normalised_min) / (t_max-t_min)

def transform_to_riskfree_rate(mu_1):
    """ Transform normalised variable to the risk-free rate. """
    return transform_ab_to_cd(mu_1, normalised_min, normalised_max,
                                    riskfree_rate_min, riskfree_rate_max)

def transform_to_volatility(mu_2):
    """ Transform normalised variable to the volatility. """
    return transform_ab_to_cd(mu_2, normalised_min, normalised_max,
                                    volatility_min, volatility_max)
    
def transform_to_correlation(mu_3):
    """ Transform normalised variable to the correlation. """
    return transform_ab_to_cd(mu_3, normalised_min, normalised_max,
                                    correlation_min, correlation_max)

def normalise_riskfree_rate(riskfree_rate):
    """ Transform risk-free rate to its corresponding normalised variable. """
    return transform_ab_to_cd(riskfree_rate,
                              riskfree_rate_min, riskfree_rate_max, 
                              normalised_min, normalised_max)
    
def normalise_volatility(volatility):
    """ Transform volatility to its corresponding normalised variable. """
    return transform_ab_to_cd( volatility, volatility_min, volatility_max,
                                            normalised_min, normalised_max)
    
def normalise_correlation(correlation):
    """ Transform correlation to its corresponding normalised variable. """
    return transform_ab_to_cd(correlation, correlation_min, correlation_max,
                                            normalised_min, normalised_max)


riskfree_rate_eval_normalised = normalise_riskfree_rate(riskfree_rate_eval)
volatility1_eval_normalised = normalise_volatility(volatility1_eval)
volatility2_eval_normalised = normalise_volatility(volatility2_eval)
correlation_eval_normalised = normalise_correlation(correlation_eval)

In [6]:
class DenseLayer(keras.layers.Layer):
    """ Define one layer of the dense network. """

    def __init__(self, units=90, input_dim=90):
        """ Construct the layer by creating all weights and biases in keras. """
        super(DenseLayer, self).__init__()
        self.units = units

        # create all weights and biases
        self.W = self.add_weight("W", shape=(input_dim, self.units),
                                 initializer="random_normal", trainable=True)
        self.b = self.add_weight("b", shape=(self.units,),
                                 initializer="random_normal", trainable=True)

    def call(self, input_combined):
        """ Returns the result of the layer calculation.

        Keyword arguments:
        input_combined -- Dictionary containing the original input of
        the neural network as 'original_variable' and
        the output of the previous layer as 'previous_layer'.
        """
        original_variable = input_combined['original_variable']
        
        # Evaluate one layer using the weights created by the constructor
        output = tf.matmul(original_variable, self.W) + self.b
        output = tf.keras.activations.tanh(output)
        return output

In [7]:
def create_network(inputs):
        # Layer 0: Dense layer of 90 nodes
    layer0 = DenseLayer(units=90, input_dim=dimension_total)
    outputs_layer0 = layer0({'original_variable': inputs})

    # Block 1: 8x Dense Layer of 90 nodes + Output of Layer 0
    layer1_1 = DenseLayer()
    outputs_layer1_1 = layer1_1({'original_variable': outputs_layer0})
    layer1_2 = DenseLayer()
    outputs_layer1_2 = layer1_2({'original_variable': outputs_layer1_1})
    # ... similarly for layer1_3 to layer1_8
    layer1_3 = DenseLayer()
    outputs_layer1_3 = layer1_3({'original_variable': outputs_layer1_2})
    layer1_4 = DenseLayer()
    outputs_layer1_4 = layer1_4({'original_variable': outputs_layer1_3})
    layer1_5 = DenseLayer()
    outputs_layer1_5 = layer1_5({'original_variable': outputs_layer1_4})
    layer1_6 = DenseLayer()
    outputs_layer1_6 = layer1_6({'original_variable': outputs_layer1_5})
    layer1_7 = DenseLayer()
    outputs_layer1_7 = layer1_7({'original_variable': outputs_layer1_6})
    layer1_8 = DenseLayer()
    outputs_layer1_8 = layer1_8({'original_variable': outputs_layer1_7})
    outputs_block1 = outputs_layer0 + outputs_layer1_8

    # Block 2: 8x Dense Layer of 90 nodes + Output of Block 1
    layer2_1 = DenseLayer()
    outputs_layer2_1 = layer2_1({'original_variable': outputs_block1})
    layer2_2 = DenseLayer()
    outputs_layer2_2 = layer2_2({'original_variable': outputs_layer2_1})
    # ... similarly for layer2_3 to layer2_8
    layer2_3 = DenseLayer()
    outputs_layer2_3 = layer2_3({'original_variable': outputs_layer2_2})
    layer2_4 = DenseLayer()
    outputs_layer2_4 = layer2_4({'original_variable': outputs_layer2_3})
    layer2_5 = DenseLayer()
    outputs_layer2_5 = layer2_5({'original_variable': outputs_layer2_4})
    layer2_6 = DenseLayer()
    outputs_layer2_6 = layer2_6({'original_variable': outputs_layer2_5})
    layer2_7 = DenseLayer()
    outputs_layer2_7 = layer2_7({'original_variable': outputs_layer2_6})
    layer2_8 = DenseLayer()
    outputs_layer2_8 = layer2_8({'original_variable': outputs_layer2_7})
    outputs_block2 = outputs_block1 + outputs_layer2_8

    # Block 3: 8x Dense Layer of 90 nodes + Output of Block 2
    layer3_1 = DenseLayer()
    outputs_layer3_1 = layer3_1({'original_variable': outputs_block2})
    layer3_2 = DenseLayer()
    outputs_layer3_2 = layer3_2({'original_variable': outputs_layer3_1})
    # ... similarly for layer3_3 to layer3_8
    layer3_3 = DenseLayer()
    outputs_layer3_3 = layer3_3({'original_variable': outputs_layer3_2})
    layer3_4 = DenseLayer()
    outputs_layer3_4 = layer3_4({'original_variable': outputs_layer3_3})
    layer3_5 = DenseLayer()
    outputs_layer3_5 = layer3_5({'original_variable': outputs_layer3_4})
    layer3_6 = DenseLayer()
    outputs_layer3_6 = layer3_6({'original_variable': outputs_layer3_5})
    layer3_7 = DenseLayer()
    outputs_layer3_7 = layer3_7({'original_variable': outputs_layer3_6})
    layer3_8 = DenseLayer()
    outputs_layer3_8 = layer3_8({'original_variable': outputs_layer3_7})
    outputs_block3 = outputs_block2 + outputs_layer3_8
    
    # Block 4: 8x Dense Layer of 90 nodes + Output of Block 3
    layer4_1 = DenseLayer()
    outputs_layer4_1 = layer4_1({'original_variable': outputs_block3})
    layer4_2 = DenseLayer()
    outputs_layer4_2 = layer4_2({'original_variable': outputs_layer4_1})
    layer4_3 = DenseLayer()
    outputs_layer4_3 = layer4_3({'original_variable': outputs_layer4_2})
    layer4_4 = DenseLayer()
    outputs_layer4_4 = layer4_4({'original_variable': outputs_layer4_3})
    layer4_5 = DenseLayer()
    outputs_layer4_5 = layer4_5({'original_variable': outputs_layer4_4})
    layer4_6 = DenseLayer()
    outputs_layer4_6 = layer4_6({'original_variable': outputs_layer4_5})
    layer4_7 = DenseLayer()
    outputs_layer4_7 = layer4_7({'original_variable': outputs_layer4_6})
    layer4_8 = DenseLayer()
    outputs_layer4_8 = layer4_8({'original_variable': outputs_layer4_7})
    outputs_block4 = outputs_block3 + outputs_layer4_8
    
        # Block 5: 8x Dense Layer of 90 nodes + Output of Block 4
    layer5_1 = DenseLayer()
    outputs_layer5_1 = layer5_1({'original_variable': outputs_block4})
    layer5_2 = DenseLayer()
    outputs_layer5_2 = layer5_2({'original_variable': outputs_layer5_1})
    layer5_3 = DenseLayer()
    outputs_layer5_3 = layer5_3({'original_variable': outputs_layer5_2})
    layer5_4 = DenseLayer()
    outputs_layer5_4 = layer5_4({'original_variable': outputs_layer5_3})
    layer5_5 = DenseLayer()
    outputs_layer5_5 = layer5_5({'original_variable': outputs_layer5_4})
    layer5_6 = DenseLayer()
    outputs_layer5_6 = layer5_6({'original_variable': outputs_layer5_5})
    layer5_7 = DenseLayer()
    outputs_layer5_7 = layer5_7({'original_variable': outputs_layer5_6})
    layer5_8 = DenseLayer()
    outputs_layer5_8 = layer5_8({'original_variable': outputs_layer5_7})
    outputs_block5 = outputs_block4 + outputs_layer5_8

    # Block 6: 8x Dense Layer of 90 nodes + Output of Block 5
    layer6_1 = DenseLayer()
    outputs_layer6_1 = layer6_1({'original_variable': outputs_block5})
    layer6_2 = DenseLayer()
    outputs_layer6_2 = layer6_2({'original_variable': outputs_layer6_1})
    layer6_3 = DenseLayer()
    outputs_layer6_3 = layer6_3({'original_variable': outputs_layer6_2})
    layer6_4 = DenseLayer()
    outputs_layer6_4 = layer6_4({'original_variable': outputs_layer6_3})
    layer6_5 = DenseLayer()
    outputs_layer6_5 = layer6_5({'original_variable': outputs_layer6_4})
    layer6_6 = DenseLayer()
    outputs_layer6_6 = layer6_6({'original_variable': outputs_layer6_5})
    layer6_7 = DenseLayer()
    outputs_layer6_7 = layer6_7({'original_variable': outputs_layer6_6})
    layer6_8 = DenseLayer()
    outputs_layer6_8 = layer6_8({'original_variable': outputs_layer6_7})
    outputs_block6 = outputs_block5 + outputs_layer6_8

    


    # Output Layer: Dense layer of 1 node
    last_layer = keras.layers.Dense(1)
    outputs_dnn = last_layer(outputs_block6)
    
    # Remaining part of the network that was provided before
    inputs_t_normalised = inputs[:, 0:1]
    inputs_x1_normalised = inputs[:, 1:2]
    inputs_x2_normalised = inputs[:, 2:3]
    inputs_p1_normalised = inputs[:, 3:4]
    
    inputs_t = transform_to_time(inputs_t_normalised)
    inputs_x1 = transform_to_logprice(inputs_x1_normalised)
    inputs_x2 = transform_to_logprice(inputs_x2_normalised)
    inputs_s_mean = (tf.math.exp(inputs_x1) + tf.math.exp(inputs_x2))/2.
    riskfree_rate = transform_to_riskfree_rate(inputs_p1_normalised)

    localisation = tf.math.log(1+tf.math.exp(localisation_parameter * (
            inputs_s_mean - strike_price * tf.exp( - riskfree_rate * inputs_t)
              )))/localisation_parameter

    return outputs_dnn + localisation

In [8]:
class DPDEGenerator(keras.utils.Sequence):
    """ Create batches of random points for the network training. """

    def __init__(self, batch_size):
        """ Initialise the generator by saving the batch size. """
        self.batch_size = batch_size
      
    def __len__(self):
        """ Describes the number of points to create """
        return self.batch_size
    
    def __getitem__(self, idx):
        """ Get one batch of random points in the interior of the domain to 
        train the PDE residual and with initial time to train the initial value.
        """
        data_train_interior = np.random.uniform(
            normalised_min, normalised_max, [self.batch_size, dimension_total]) 

        t_train_initial = normalised_min * np.ones((self.batch_size, 1))
        s_and_p_train_initial = np.random.uniform(
            normalised_min, normalised_max,
            [self.batch_size, dimension_state + dimension_parameter])
        
        data_train_initial = np.concatenate(
            (t_train_initial, s_and_p_train_initial), axis=1)

        return [data_train_interior, data_train_initial]

In [9]:
class DPDEModel(keras.Model):
    """ Create a keras model with the deep param. PDE loss function """

    def train_step(self, data):
        """ Create one optimisation stop based on the deep param. PDE loss function. """
        data_interior, data_initial = data

        riskfree_rate_interior = transform_to_riskfree_rate(
            data_interior[:, 3:4])
        volatility1_interior = transform_to_volatility(data_interior[:, 4:5])
        volatility2_interior = transform_to_volatility(data_interior[:, 5:6])
        correlation_interior = transform_to_correlation(data_interior[:, 6:7])

        x1_initial = transform_to_logprice(data_initial[:, 1:2])
        x2_initial = transform_to_logprice(data_initial[:, 2:3])

        with tf.GradientTape() as tape:
            v_interior = self(data_interior, training=True)  # Forward pass
            v_initial = self(data_initial, training=True)  # Forward pass bdry

            gradient = K.gradients(v_interior, data_interior)[0]

            v_dt = diff_dt * gradient[:, 0:1]
            v_dx1 = diff_dx * gradient[:, 1:2]
            v_dx2 = diff_dx * gradient[:, 2:3]

            grad_v_dx1 = K.gradients(v_dx1, data_interior)[0]
            grad_v_dx2 = K.gradients(v_dx2, data_interior)[0]

            v_dx1dx1 = diff_dx * grad_v_dx1[:, 1:2]
            v_dx2dx2 = diff_dx * grad_v_dx2[:, 2:3]
            v_dx1dx2 = diff_dx * grad_v_dx1[:, 2:3]
            v_dx2dx1 = diff_dx * grad_v_dx2[:, 1:2]

            residual_interior = (
                v_dt + riskfree_rate_interior * v_interior 
                - (riskfree_rate_interior - volatility1_interior**2/2) * v_dx1
                - (riskfree_rate_interior - volatility2_interior**2/2) * v_dx2
                - 0.5 * volatility1_interior**2 * v_dx1dx1
                - 0.5 * volatility2_interior**2 * v_dx2dx2 
                - 0.5 * correlation_interior 
                    * volatility1_interior * volatility2_interior * v_dx1dx2 
                - 0.5 * correlation_interior 
                    * volatility2_interior * volatility1_interior * v_dx2dx1
                )

            s_mean_initial = 0.5 * (
                tf.math.exp(x1_initial)+tf.math.exp(x2_initial)) 
            payoff_initial = K.maximum(s_mean_initial - strike_price, 0)

            loss_interior = K.mean(K.square(residual_interior))
            loss_initial = K.mean(K.square(v_initial - payoff_initial))
            
            loss = loss_initial + loss_interior

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        return {"loss": loss, 
                "loss initial": loss_initial, 
                "loss interior": loss_interior}

In [ ]:
%%time

    # Create and train model from scratch. 
    inputs = keras.Input(shape=(dimension_total,))
    outputs = create_network(inputs)
    model = DPDEModel(inputs=inputs, outputs=outputs)
    batch_generator = DPDEGenerator(n_train)
    model.compile(optimizer=tf.keras.optimizers.Adam(initial_learning_rate))
    callback = tf.keras.callbacks.EarlyStopping(
        'loss', patience=50, restore_best_weights=True)
    
    model.fit(x=batch_generator, epochs=nr_epochs, steps_per_epoch=10,
                          callbacks=[callback])

Epoch 1/601
 8/10 [=======================>......] - ETA: 2s - loss: 2.8148 - loss initial: 2.2775 - loss interior: 0.5373

In [ ]:
def decompose_covariance_matrix(t, volatility1, volatility2, correlation):
    """ Decompose covariance matrix as in Lemma 3.1 of Bayer et. al (2018). """
    sigma_det = (1-correlation**2) * volatility1**2 * volatility2**2
    sigma_sum = (volatility1**2 + volatility2**2 
                  - 2*correlation*volatility1*volatility2)

    ev1 = volatility1**2 - correlation*volatility1*volatility2
    ev2 = -(volatility2**2 - correlation*volatility1*volatility2)
    ev_norm = np.sqrt(ev1**2 + ev2**2)

    eigenvalue = volatility1**2 + volatility2**2 - 2*sigma_det/sigma_sum

    v_mat = np.array([ev1, ev2]) / ev_norm
    d = t * np.array([sigma_det/sigma_sum, eigenvalue])
    return d, v_mat

def one_dimensional_exact_solution(
        t, s, riskfree_rate, volatility, strike_price):
    """ Standard Black-Scholes formula """

    d1 = (1 / (volatility*np.sqrt(t))) * (
            np.log(s/strike_price) 
            + (riskfree_rate + volatility**2/2.) * t
        )
    d2 = d1 - volatility*np.sqrt(t)
    return (norm.cdf(d1) * s 
            - norm.cdf(d2) * strike_price * np.exp(-riskfree_rate*t))

def exact_solution(
    t, s1, s2, riskfree_rate, volatility1, volatility2, correlation):
    """ Compute the option price of a European basket call option. """
    if t == 0:
        return np.maximum(0.5*(s1+s2) - strike_price, 0)

    d, v = decompose_covariance_matrix(
        t, volatility1, volatility2, correlation)
    
    beta = [0.5 * s1 * np.exp(-0.5*t*volatility1**2),
            0.5 * s2 * np.exp(-0.5*t*volatility2**2)]
    integration_points, integration_weights = hermgauss(33)

    # Transform points and weights
    integration_points = np.sqrt(2*d[1]) * integration_points.reshape(-1, 1)
    integration_weights = integration_weights.reshape(1, -1) / np.sqrt(np.pi)

    h_z = (beta[0] * np.exp(v[0]*integration_points)
           + beta[1] * np.exp(v[1]*integration_points))

    evaluation_at_integration_points = one_dimensional_exact_solution(
        t=1, s=h_z * np.exp(0.5*d[0]), 
        strike_price=np.exp(-riskfree_rate * t) * strike_price, 
        volatility=np.sqrt(d[0]), riskfree_rate=0.
        )
    
    solution = np.matmul(integration_weights, evaluation_at_integration_points)
    
    return solution[0, 0]

test_solution = exact_solution(t=4., s1=100., s2=100., riskfree_rate=0.2, 
               volatility1=0.1, volatility2=0.3, correlation=0.5)
assert(np.abs(test_solution - 55.096796282039364) < 1e-10)

In [ ]:
def localisation(t, s1, s2, riskfree_rate=riskfree_rate_eval):
    """ Return the value of the localisation used in the network. """
    return 1/localisation_parameter * np.log(1 +
                    np.exp(localisation_parameter * (
                        0.5*(s1+s2) - np.exp(-riskfree_rate*t)*strike_price))
                    )

In [ ]:
def get_random_points_of_interest(nr_samples, 
                    t_min_interest=t_min_interest,
                    t_max_interest=t_max_interest,
                    s_min_interest=s_min_interest,
                    s_max_interest=s_max_interest,
                    parameter_min_interest_normalised=normalised_min,
                    parameter_max_interest_normalised=normalised_max):
    """ Get a number of random points within the defined domain of interest. """
    t_sample = np.random.uniform(t_min_interest, t_max_interest, 
                                 [nr_samples, 1])
    t_sample_normalised = normalise_time(t_sample)

    s_sample = np.random.uniform(
        s_min_interest, s_max_interest, [nr_samples, dimension_state])
    s1_sample = s_sample[:, 0:1]
    s2_sample = s_sample[:, 1:2]
    x_sample_normalised = normalise_logprice(np.log(s_sample))

    parameter_sample_normalised = np.random.uniform(
        normalised_min, normalised_max, [nr_samples, dimension_parameter])
    data_normalised = np.concatenate(
        (t_sample_normalised, x_sample_normalised, parameter_sample_normalised),
        axis=1
        )

    riskfree_rate_sample = transform_to_riskfree_rate(
        parameter_sample_normalised[:, 0])
    volatility1_sample = transform_to_volatility(
        parameter_sample_normalised[:, 1])
    volatility2_sample = transform_to_volatility(
        parameter_sample_normalised[:, 2])
    correlation_sample = transform_to_correlation(
        parameter_sample_normalised[:, 3])
    
    return data_normalised, t_sample.reshape(-1), s1_sample.reshape(-1), \
            s2_sample.reshape(-1), riskfree_rate_sample, volatility1_sample, \
            volatility2_sample, correlation_sample


def get_points_for_plot_at_fixed_time(t_fixed=t_max,
                s_min_interest=s_min_interest, s_max_interest=s_max_interest,
                riskfree_rate_fixed=riskfree_rate_eval,
                volatility1_fixed=volatility1_eval,
                volatility2_fixed=volatility2_eval,
                correlation_fixed=correlation_eval,
                n_plot=nr_samples_surface_plot):
    """ Get the spacial and normalised values for surface plots 
    at fixed time and parameter, varying both asset prices. 
    """
    s1_plot = np.linspace(s_min_interest, s_max_interest, n_plot).reshape(-1,1)
    s2_plot = np.linspace(s_min_interest, s_max_interest, n_plot).reshape(-1,1)
    [s1_plot_mesh, s2_plot_mesh] = np.meshgrid(s1_plot, s2_plot, indexing='ij')

    x1_plot_mesh_normalised = normalise_logprice(
        np.log(s1_plot_mesh)).reshape(-1,1)

    x2_plot_mesh_normalised = normalise_logprice(
        np.log(s2_plot_mesh)).reshape(-1,1)

    t_mesh = t_fixed  * np.ones((n_plot**2, 1))
    t_mesh_normalised = normalise_time(t_mesh)

    parameter1_mesh_normalised = (normalise_riskfree_rate(riskfree_rate_fixed) 
                                                      * np.ones((n_plot**2, 1)))
    parameter2_mesh_normalised = (normalise_volatility(volatility1_fixed) 
                                                      * np.ones((n_plot**2, 1)))
    parameter3_mesh_normalised = (normalise_volatility(volatility2_fixed) 
                                                      * np.ones((n_plot**2, 1)))
    parameter4_mesh_normalised = (normalise_correlation(correlation_fixed) 
                                                      * np.ones((n_plot**2, 1)))

    x_plot_normalised = np.concatenate((t_mesh_normalised,
                                        x1_plot_mesh_normalised,
                                        x2_plot_mesh_normalised,
                                        parameter1_mesh_normalised, 
                                        parameter2_mesh_normalised,
                                        parameter3_mesh_normalised, 
                                        parameter4_mesh_normalised), axis=1)

    
    return s1_plot_mesh, s2_plot_mesh, x_plot_normalised


s1_plot_mesh, s2_plot_mesh, x_plot_normalised = \
    get_points_for_plot_at_fixed_time()

In [ ]:
DPDE_solution = model.predict(x_plot_normalised).reshape(
    nr_samples_surface_plot, nr_samples_surface_plot)

exact_solution_evaluated = [exact_solution(t=t_max, s1=s1[0], s2=s2[0], 
                                riskfree_rate=riskfree_rate_eval, 
                                volatility1=volatility1_eval, 
                                volatility2=volatility2_eval,
                                correlation=correlation_eval)
                  for s1, s2 in zip(
                      s1_plot_mesh.reshape(-1, 1), s2_plot_mesh.reshape(-1, 1))
                  
                  ]
exact_solution_evaluated = np.array(exact_solution_evaluated)
exact_solution_evaluated = exact_solution_evaluated.reshape(
    nr_samples_surface_plot, nr_samples_surface_plot)

localisation_plot = localisation(4., s1_plot_mesh, s2_plot_mesh, riskfree_rate_eval)

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_surface(s1_plot_mesh, s2_plot_mesh, DPDE_solution, cmap='viridis')
ax.set_title('DPDE Solution')
ax.set_xlabel('$s_1$')
ax.set_ylabel('$s_2$')
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

error = np.abs(DPDE_solution - exact_solution_evaluated)
ax.plot_surface(s1_plot_mesh, s2_plot_mesh, error, cmap='viridis')
ax.set_title('Error')
ax.set_xlabel('$s_1$')
ax.set_ylabel('$s_2$')
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

dnn_part = DPDE_solution - localisation_plot
ax.plot_surface(s1_plot_mesh, s2_plot_mesh, dnn_part, cmap='viridis')
ax.set_title('DNN part')
ax.set_xlabel('$s_1$')
ax.set_ylabel('$s_2$')
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_surface(s1_plot_mesh, s2_plot_mesh, localisation_plot, cmap='viridis')
ax.set_title('Localisation')
ax.set_xlabel('$s_1$')
ax.set_ylabel('$s_2$')
plt.show()

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_surface(s1_plot_mesh, s2_plot_mesh, exact_solution_evaluated, cmap='viridis')
ax.set_title('Exact solution')
ax.set_xlabel('$s_1$')
ax.set_ylabel('$s_2$')
plt.show()

In [ ]:
data_samples, t_samples, s1_samples, s2_samples, riskfree_rate_samples, \
  volatility1_samples, volatility2_samples, correlation_samples = \
              get_random_points_of_interest(nr_samples_scatter_plot)

In [ ]:
print('Predict {} values and measure the time:'.format(nr_samples_scatter_plot))
%time DPDE_solution = model.predict(data_samples)

In [ ]:
exact_solution_evaluated = [exact_solution(t=t, s1=s1, s2=s2,
                                  riskfree_rate=riskfree_rate,
                                  volatility1=volatility1,
                                  volatility2=volatility2,
                                  correlation=correlation
                                  ) 
                  for t, s1, s2, riskfree_rate, volatility1, volatility2,
                      correlation 
                  in zip(t_samples, s1_samples, s2_samples, riskfree_rate_samples, 
                          volatility1_samples, volatility2_samples,
                          correlation_samples)]

exact_solution_evaluated = np.array(exact_solution_evaluated).reshape(-1,1)

In [ ]:
plt.scatter(exact_solution_evaluated, DPDE_solution)
plt.xlabel('Exact Solution')
plt.ylabel('DPDE Solution')
plt.show()

In [ ]:
plt.scatter(exact_solution_evaluated, 
            np.abs(exact_solution_evaluated - DPDE_solution))
plt.xlabel('Exact Solution')
plt.ylabel('DPDE Error')
plt.show()

In [ ]:
data_samples, t_samples, s1_samples, s2_samples, riskfree_rate_samples, \
  volatility1_samples, volatility2_samples, correlation_samples = \
              get_random_points_of_interest(nr_samples_error_calculation)

In [ ]:
print('Predict {} values and measure the time:'.format(nr_samples_error_calculation))
%time DPDE_solution = model.predict(data_samples)

In [ ]:
exact_solution_evaluated = [exact_solution(t=t, s1=s1, s2=s2, 
                                  riskfree_rate=riskfree_rate,
                                  volatility1=volatility1,
                                  volatility2=volatility2,
                                  correlation=correlation) 
            for t, s1, s2, riskfree_rate, volatility1, volatility2, correlation 
            in zip(t_samples, s1_samples, s2_samples, riskfree_rate_samples, 
                    volatility1_samples, volatility2_samples,
                    correlation_samples)]

exact_solution_evaluated = np.array(exact_solution_evaluated).reshape(-1, 1)

In [ ]:
print('Estimated MSE Error:')
print(np.sqrt(np.mean(np.square(exact_solution_evaluated - DPDE_solution))))

print('Relative Error to L2 Norm in %:')
print(np.sqrt(np.mean(np.square(exact_solution_evaluated - DPDE_solution))) 
        / np.sqrt(np.mean(np.square(exact_solution_evaluated)))*100)

print('Maximal Error:')
print(np.max(exact_solution_evaluated - DPDE_solution))

In [ ]:
print('Estimated MSE Error:')
print(0.030773506093868433)
print('Relative Error to L2 Norm in %:')
print(0.08530248578852807)
print('Maximal Error:')
print(0.1987940127602612)

In [ ]:
model.summary()